In [1]:
import matplotlib.pyplot as plt
import json
import os
import cv2
import time
import numpy as np
from csbdeep.utils import Path, normalize
from segmentation import MELC_Segmentation
import pickle
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
import umap


2023-06-07 15:24:42.629330: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-07 15:24:42.630534: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 15:24:42.655766: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-07 15:24:42.656182: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-07 15:24:43.026429: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
data = "ALS"

f = open('/data_slow/je30bery/spatial_proteomics/config.json')
config = json.load(f)
data_path = config[data]
seg = MELC_Segmentation(data_path, membrane_marker="cd45") 
print(seg.fields_of_view)

['ALS01 - 21297', 'ALS01 - 21298', 'ALS01 - 21299', 'ALS01 - 21300', 'ALS02 - 21301', 'ALS02 - 21302', 'ALS02 - 21303', 'ALS02 - 21304', 'ALS03 - 21305', 'ALS03 - 21306', 'ALS03 - 21307', 'ALS03 - 21308', 'Healthy01 - 21315', 'Healthy01 - 21316', 'Healthy01 - 21317', 'Healthy01 - 21318', 'Healthy02 - 21319', 'Healthy02 - 21320', 'Healthy02 - 21321', 'Healthy02 - 21322', 'Healthy03 - 21323', 'Healthy03 - 21324', 'Healthy03 - 21325', 'Healthy03 - 21326', 'Healthy04 - 21335', 'Healthy04 - 21336', 'Healthy04 - 21337', 'Healthy04 - 21338', 'Healthy05 - 21331', 'Healthy05 - 21332', 'Healthy05 - 21333', 'Healthy05 - 21334']


In [ ]:
# Save segmentation results
if 0:
    for i, fov in enumerate(seg.fields_of_view):
        seg.field_of_view = fov
        nuc, mem = seg.run(fov)
        np.save(f"./segmentation_results/{fov}_nuclei.npy", nuc.astype(int))
        np.save(f"./segmentation_results/{fov}_membranes.npy", mem.astype(int))

        with open(f'./segmentation_results/{fov}_nuclei.pickle', 'wb') as handle:
            pickle.dump(seg.nucleus_label_where[fov], handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(f'./segmentation_results/{fov}_membrane.pickle', 'wb') as handle:
            pickle.dump(seg.membrane_label_where[fov], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
def get_expression(adaptive, where_dict):
    expression = np.zeros_like(adaptive)
    expression_dict = dict()
    for n in where_dict:
        if n == 0:
            continue
        
        segment = where_dict[n]
        exp = np.sum(adaptive[segment[0], segment[1]])/len(segment[0])
        expression[segment[0], segment[1]] = exp
        expression_dict[n] = exp / 255
    return expression_dict #expression, 

In [ ]:
# calculate expression within nuclei
if 0:
    for fov in seg.fields_of_view:
        with open( f"./segmentation_results/{fov}_nuclei.pickle", 'rb') as handle:
            where_dict_nuc = pickle.load(handle)

        seg.field_of_view = fov
        markers = {m.split("_")[1]: os.path.join(seg.get_fov_dir(), m) for m in sorted(os.listdir(seg.get_fov_dir())) if m.endswith(".tif") and "phase" not in m}
        del markers['Propidium iodide']
        rows = list(where_dict_nuc.keys())
        cols = list(markers.keys())
        result_df = pd.DataFrame(index=rows, columns=markers)

        for m in markers:
            m_img = cv2.imread(markers[m], cv2.IMREAD_GRAYSCALE) 
            tile_std = np.std(m_img)
            adaptive = cv2.adaptiveThreshold(m_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 201, -tile_std)  
            expression_dict = get_expression(adaptive, where_dict_nuc)
            assert list(where_dict_nuc.keys()) == list(expression_dict.keys())
            result_df[m] = list(expression_dict.values())
        result_df.to_pickle(f"./marker_expression_nuclei_results/{fov}.pkl")  

In [ ]:
# calculate expression within entire cells
if 0:
    for fov in seg.fields_of_view:
        with open( f"./segmentation_results/{fov}_membrane.pickle", 'rb') as handle:
            where_dict_nuc = pickle.load(handle)

        seg.field_of_view = fov
        markers = {m.split("_")[1]: os.path.join(seg.get_fov_dir(), m) for m in sorted(os.listdir(seg.get_fov_dir())) if m.endswith(".tif") and "phase" not in m}
        del markers['Propidium iodide']

        rows = list(where_dict_nuc.keys())
        cols = list(markers.keys())
        result_df = pd.DataFrame(index=rows, columns=markers)

        for m in markers:
            m_img = cv2.imread(markers[m], cv2.IMREAD_GRAYSCALE) 
            tile_std = np.std(m_img)
            adaptive = cv2.adaptiveThreshold(m_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 201, -tile_std)  
            expression_dict = get_expression(adaptive, where_dict_nuc)
            assert list(where_dict_nuc.keys()) == list(expression_dict.keys())
            result_df[m] = list(expression_dict.values())

        result_df.to_pickle(f"./marker_expression_membrane_results/{fov}.pkl")  

In [ ]:
import seaborn as sns
import pandas as pd

pal = sns.color_palette("rocket", 3) + sns.color_palette("mako", 5)
index = result1.keys()
#vals0 = result0.values()
vals1 = result1.values()
#vals2 = result2.values()
sns.set(rc={'figure.figsize':(5,4)})

#df = pd.DataFrame(np.array([list(index), list(vals0), list(vals1), list(vals2)]).T, columns=["ID", "Condition cells 0","Condition cells 1", "Condition cells 2"])
df = pd.DataFrame(np.array([list(index), list(vals1)]).T, columns=["ID", "Condition cells 1"])

df["Group"] = np.where(df["ID"].str.contains("ALS"), "ALS", "Control")
df["Patient"] = df["ID"].apply(lambda x: x.split(" ")[0])

#df["Condition cells 0"] = df["Condition cells 0"].astype(float) * 100
df["Condition cells 1"] = df["Condition cells 1"].astype(float) * 100
#df["Condition cells 2"] = df["Condition cells 2"].astype(float) * 100

In [ ]:
plt.clf()
sns.set_theme()
#df = pd.read_csv("nuclei_results.csv")

sns.scatterplot(data=df, x="Group", y="Condition cells 1", hue="Patient", palette=pal)
plt.tight_layout()
plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel("[%] Cells that show profile") #TODO
#plt.title("Profile CD11b:0, CD16:1, CD45-RA:1, HLA-DR:0, Expression within Cells (Nuclei and Cytoplasm)") #TODO
plt.title("Profile CD11b:0, CD16:1, HLA-DQ:0, Expression within Cells (Nuclei and Cytoplasm)") #TODO

#plt.title("Profile CD11b:0, CD16:1, CD45-RA:1, HLA-DR:0, Expression within Nuclei") #TODO
#plt.title("Profile CD11b:0, CD16:1, HLA-DQ:0, Expression within Nuclei") #TODO
plt.savefig("expr_mem_1.png", bbox_inches='tight') #TODO